In [3]:
import reportlab
from reportlab.pdfgen import canvas
from reportlab.lib.units import inch

def create_grid_pdf(filename, page_width, page_height, num_horizontal_grids, num_vertical_grids,
                   horizontal_spacing, vertical_spacing, line_width=0.5, line_color="#cccccc"):
    """
    Creates a PDF file with a configurable grid layout.

    Args:
        filename (str): The desired filename for the PDF file.
        page_width (float): The width of the PDF page in inches.
        page_height (float): The height of the PDF page in inches.
        num_horizontal_grids (int): The number of horizontal grid lines.
        num_vertical_grids (int): The number of vertical grid lines.
        horizontal_spacing (float): The spacing between horizontal grid lines in inches.
        vertical_spacing (float): The spacing between vertical grid lines in inches.
        line_width (float, optional): The width of the grid lines in points (1/72 inch). Defaults to 0.5.
        line_color (str, optional): The color of the grid lines. Defaults to "#cccccc" (light gray).
    """

    # Calculate grid coordinates based on spacing and number of grids
    y_coords = [page_height - i * vertical_spacing for i in range(num_vertical_grids + 1)]
    x_coords = [i * horizontal_spacing for i in range(num_horizontal_grids + 1)]

    # Create a new PDF document
    c = canvas.Canvas(filename, pagesize=(page_width, page_height))

    # # Draw horizontal grid lines
    # for y in y_coords:
    #     c.line(0, y, page_width, y, stroke=line_color, linewidth=line_width)

    # # Draw vertical grid lines
    # for x in x_coords:
    #     c.line(x, 0, x, page_height, stroke=line_color, linewidth=line_width)

    # Draw horizontal grid lines
    for y in y_coords:
        c.line(0, y, page_width, y, color=line_color, linewidth=line_width)  # Use 'color' instead of 'stroke'

    # Draw vertical grid lines
    for x in x_coords:
        c.line(x, 0, x, page_height, color=line_color, linewidth=line_width)  # Use 'color' instead of 'stroke'

    # Save the PDF
    c.save()

# Example usage:
filename = "grid_page.pdf"
page_width = 8.5  # Standard letter paper width in inches
page_height = 11  # Standard letter paper height in inches
num_horizontal_grids = 4
num_vertical_grids = 6
horizontal_spacing = 1  # Customizable horizontal spacing in inches
vertical_spacing = 1.5  # Customizable vertical spacing in inches

create_grid_pdf(filename, page_width, page_height, num_horizontal_grids, num_vertical_grids,
               horizontal_spacing, vertical_spacing)

print(f"Grid PDF created: {filename}")


TypeError: line() got an unexpected keyword argument 'color'